Staging Layer

In [9]:
import pandas as pd
employee = pd.read_csv('../Source/employee.csv')
course = pd.read_csv('../Source/courses.csv')
employee_course = pd.read_csv('../Source/employee_course.csv')

In [16]:
employee_df = pd.DataFrame(employee)
course_df = pd.DataFrame(course)
employee_course_df = pd.DataFrame(employee_course)

In [17]:
print(employee.info())
print(course.info())
print(employee_course_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   empID             500 non-null    object
 1   name              500 non-null    object
 2   designation       500 non-null    object
 3   password          500 non-null    object
 4   email             500 non-null    object
 5   mobile            500 non-null    int64 
 6   performance_rate  500 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 27.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   courseID          50 non-null     object
 1   name              50 non-null     object
 2   duration          50 non-null     object
 3   difficulty_level  50 non-null     object
 4   learning_path     50 non-null     object
dt

In [22]:
employee_df.to_csv('Staging_Layer/employee.csv', index=False)
course_df.to_csv('Staging_Layer/course.csv', index=False)
employee_course_df.to_csv('Staging_Layer/employee_course.csv',index=False)

Mart Layer